In [1]:
from collections import defaultdict
import os
from pathlib import Path
from pprint import pprint
import re

import yaml

# Define the directory path and search pattern
directory_path = "results/2024/04.19/13-45-48"
search_text = "Exception ignored in: <function Logger.__del__"

# Find files matching the pattern
files = Path(directory_path).rglob(pattern="stderr*")

# Define a regular expression pattern to match the search text
pattern = re.compile(search_text)

broken_params = []
# Iterate through each file and search for the text
for file_path in files:
    with file_path.open("r") as file:
        for line_number, line in enumerate(file, start=1):
            if pattern.search(line):
                print(
                    f"Found in file: {file_path}, line: {line_number}, content: {line.strip()}"
                )
                yaml_file = file_path.parent.parent / ".hydra" / "overrides.yaml"
                broken_params.append(yaml.safe_load(yaml_file.open("r")))
                break
# pprint(broken_params)

# grouped_data = defaultdict(list)

# # Iterate through the data and group it based on the first element of each sublist
# for sublist in broken_params:
#     key = sublist[0]
#     grouped_data[key].append(sublist[1:])

# # Convert defaultdict to dict
# grouped_data = dict(grouped_data)

# pprint(grouped_data)

Found in file: results/2024/04.19/13-45-48/4/cifar100_fixed_ewc/stderr-2024-04-19-16-49.txt, line: 8, content: Exception ignored in: <function Logger.__del__ at 0x14c6d9302560>
Found in file: results/2024/04.19/13-45-48/3/cifar100_fixed_ewc/stderr-2024-04-19-16-48.txt, line: 8, content: Exception ignored in: <function Logger.__del__ at 0x1490da6ea560>


In [2]:
import pandas as pd
import numpy as np

params = np.asarray(broken_params)


def delete_to_pandas(name, arr):
    return map(lambda x: x.replace(name, ""), arr)


names = [
    "training.vcreg.reg_layers=",
    "training.vcreg.var_weight=",
    "training.vcreg.cov_weight=",
]
df = pd.DataFrame(
    [delete_to_pandas(name, params[:, idx]) for idx, name in enumerate(names)]
).T

In [3]:
df = df.rename(
    columns={
        0: "reg_layers",
        1: "var_weight",
        2: "cov_weight",
    }
)

In [4]:
list(df.T.to_dict().values())

[{'reg_layers': 'fc$', 'var_weight': '1.74', 'cov_weight': '0.64'},
 {'reg_layers': 'fc$', 'var_weight': '0.64', 'cov_weight': '12.8'}]

In [19]:
len(broken_params)

11

In [1]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("tunnels-ssl/05.17")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict["test/avg_acc_tag"])

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"avg_acc_tag": summary_list, "config": config_list, "name": name_list}
)
config_df = pd.json_normalize(runs_df["config"])
df = pd.concat([runs_df.drop(columns=["config"]), config_df], axis=1)

In [12]:
condition = df["model.network"] == "no_last_relu"
df[condition][
    [
        "training.approach.name",
        "data.exemplars.num_exemplars",
        "training.vcreg.reg_layers",
        "misc.results_path",
    ]
].sort_values(
    by=[
        "training.approach.name",
        "data.exemplars.num_exemplars",
        "training.vcreg.reg_layers",
    ]
)

,training.approach.name,data.exemplars.num_exemplars,training.vcreg.reg_layers,misc.results_path
73,ewc,0,fc$,results/2024/05.17/17-05-54/0
67,ewc,0,NaN,results/2024/05.17/17-06-00/2
72,finetuning,0,fc$,results/2024/05.17/17-05-54/1
69,finetuning,0,NaN,results/2024/05.17/17-06-00/0
66,finetuning,2000,fc$,results/2024/05.17/17-05-54/3
68,finetuning,2000,NaN,results/2024/05.17/17-06-00/1
70,lwf,0,.*after_relu,results/2024/05.17/17-05-54/2
0,lwf,0,NaN,results/2024/05.17/17-06-00/3


/net/tscratch/people/plghpiotr/cl-adaptation/notebooks
